In [ ]:
import numpy as np
from driver import io_shape_dict
from driver_base import ExampleOverlay
import IPnDR
import numpy as np

driver = ExampleOverlay(
        bitfile_name="L3FNet.bit",
        io_shape_dict=io_shape_dict,
        batch_size=1,
        runtime_weight_dir="runtime_weights/",
    )

In [ ]:
data = np.load('input.py')
IPdata = IPnDR.SAI2MacPI_plus(data, angRes=7)
# Resize to the range [0, 255]
IPdata_scaled = IPdata * 255.0
# Truncate data to ensure it is in the range [0, 255]
IPdata_scaled = np.clip(IPdata_scaled, 0, 255)

# Convert the data to uint8
IPdata_uint8 = IPdata_scaled.astype(np.uint8)
# Suppose IPdata_uint8 is your uint8 array
IPdata_transposed = np.transpose(IPdata_uint8, (2, 3, 0, 1))  # Adjust the channel and batch dimension positions
# Add a quantitative dimension
IPdata_expanded = np.expand_dims(IPdata_transposed, axis=0)
# reshape into (1, 240, 240, 1, 9)
IPdata_reshaped = IPdata_expanded.reshape((1, 336, 336, 1, 9))

In [ ]:
outputfile = 'output.npy'
driver.copy_input_data_to_device(IPdata_reshaped)
driver.execute_on_buffers(asynch=True)
outputs = []
driver.copy_output_data_from_device(driver.obuf_packed[0], ind=0)
obuf_folded = driver.unpack_output(driver.obuf_packed[0], ind=0)
obuf_normal = driver.unfold_output(obuf_folded, ind=0)
out = np.transpose(obuf_normal, (0, 3, 1, 2))
out = IPnDR.Regression(-4, 4, out)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image_data = out[0, 0, :, :]
plt.imshow(image_data, cmap='viridis')
plt.colorbar()  
plt.show()